# Lattice Angle Tolerance Calculations

##### Imports

In [227]:
import numpy as np
import scipy.optimize as sp_opt
from sympy import latex

##### Constants

In [228]:
# The distance between mirrors. This will probably be a bit underneath 44.
d = 44
# the optimal angle, as determined by other code
# convert to degrees
theta_0 = 44.94 * np.pi / 180
# the diameter of the mirror:
mirror_diameter = 2
# the y tolerance
dy_tol = 0.59255872135

## Basic Calculations

I consider the two paths of the two beams with one path held constant and the other being varied based on the angle of the mirror. 

I set the variable path to be centered (i.e. the reflection's location) at the origin, and the other path is constant. Then, the points along the variable path, in for a given θ, is
$$z=r\cos(θ),  y=r\sin(θ)$$
$$y=\tan(θ)*z$$
The equation of the other one is
$$y=\tan(-θ_0) * z+d =-\tan(θ_0) * z+d  $$
for the distance between the two mirrors being d.
Solve:
$$−\tan(θ_0) * z+d=\frac{\sin(θ)}{cos(θ)} *z → z(\tan(θ_0)+tan(θ) )=d$$
$$z=\frac{d}{\tan(θ_0)+\tan(θ)},  y=\frac{(d \tan(θ))}{\tan(θ_0)+\tan(θ)})$$

# example
$$ x = \frac{1}{5}$$

In [229]:
# Calculating the position of the intersection of the two lines:
def intersection(input_theta_0, input_dtheta, input_d, input_dx):
    z = ((d + input_dx * np.tan(input_theta_0 - input_dtheta)) 
    / (np.tan(input_theta_0 + input_dtheta) 
       + np.tan(input_theta_0 - input_dtheta)))
    y = ((d + input_dx * np.tan(input_theta_0 - input_dtheta)) 
         * np.tan(input_theta_0 + input_dtheta)
    / (np.tan(input_theta_0 + input_dtheta) 
       + np.tan(input_theta_0 - input_dtheta)))
    return(z,y)

In [230]:
def angle_tolerance(input_theta_0, input_d, input_tolerance):
    def f1(p): 
        theta = p
        return(input_d * np.sqrt((1 / (np.tan(input_theta_0) 
                                            + np.tan(theta)) - 
                      1 / (2 * np.tan(input_theta_0)))**2 + 
                     (np.tan(input_theta_0) / (np.tan(input_theta_0) 
                                             + np.tan(input_theta_0)) 
                      - 1 / 2)**2 ) - input_tolerance)
    return(sp_opt.fsolve(f1, np.pi/4))

In [231]:
z_0, y_0 = intersection(theta_0, 0, d, 0)
print("The optimal intersection: (z_0, y_0) = (", z_0, ",", y_0, ")")

The optimal intersection: (z_0, y_0) = ( 22.0461250111 , 22.0 )


## Positional Tolerances

### X and Z Tolerances

We can move the mirrors around in the x and z directions directly (keeping the incident angles constant) in order to adjust the lattice position in these directions. Suppose we get a mirror of diameter $d_{mirror}$, and suppose the usable region of that mirror is $\Delta x_{mir} = \frac{4d_{mirror}}{5}$. Then, our tolerance in the x and z directions is $\Delta x_{tol} = \Delta z_{tol} = \frac{\Delta x_{mir}}{2} = \pm \frac{2d_{mirror}}{5}$

In [232]:
dx_tol = 2 * mirror_diameter / 5
dz_tol = 2 * mirror_diameter / 5
dx_mir = 4 * mirror_diameter / 5

### Calculating the Y Positional Tolerance

We can modify the y position of our lattice by just aiming the beam so that it hits the outer edge (the edge far away from the lens) of one mirror and angling it so that it hits the inner edge of the second mirror (the edge closer to the lens). With this angling, the angle of both of the incoming and outgoing beams changes such that the total trap is shifted in the vertical direction. Unfortunately, this angling also changes the z-coordinate of the intercept of these beams, meaning that changing the y-position of our lattice is coupled to changing the z-position of the lattice. This makes calculates somewhat more complicated.

The maximum angle that we can aim at is deterimined by the size of the mirror by $$Δx_{mir}=4d_{mir}/5, Δy_{mir}=d mm →Δθ=\tan^{-1}(\Delta x_{mir} / \Delta y_{mir}) = \tan^{-1}(4d_{mir}/5d)$$ I can add this  to the angle of the beams. In order to calculate the tolerance here, I will calculate the full distance in y achieved by this maximum displacement. I place the bottom, farther back than the other one, and assume that the beam hitting it comes at best angle $θ_0$  modulo the $Δθ$ in the positive sense: $θ=θ_0+Δθ_0$. The other beam is displaced in y by $d$ and in x by $4d_{mirror}/5$ and comes with the best angle $θ_0$  modulo the $Δθ$ in the negative sense: $θ=θ_0−Δθ$

Calculating the new solution:
$$x=r\cos(θ_0+Δθ),  y=r\sin(θ_0+Δθ)$$
$$y=\tan(θ_0+Δθ) * \Delta x_{mir}$$
The equation of the other one is
$$y=−\tan(θ_0−Δθ) x+d+Δx_{mir}\tan(θ_0−Δθ)$$
for the distance between the two mirrors being d.
The extra term comes from the new y intercept of it which is no longer d but $d+Δx_{mir}\tan(θ_0−Δθ)$.
Solve:
$$−\tan(θ_0−Δθ) x+d+Δx_{mir}\tan(θ_0−Δθ)=\tan(θ_0+Δθ) x$$
$$x(\tan(θ_0+Δθ)+\tan(θ_0−Δθ) )=d+Δx_{mir}\tan(θ_0−Δθ)$$
$$x=\frac{d+Δx_{mir} \tan(θ_0−Δθ)}{\tan(θ_0+Δθ)+\tan(θ_0−Δθ)},  y=\frac{(d+Δx_{mir} \tan(θ_0−Δθ) )  \tan(θ_0+Δθ)}{\tan(θ_0+Δθ)+\tan(θ_0−Δθ) }$$

In [233]:
dtheta = np.arctan(dx_mir / d)
z, y = intersection(theta_0, dtheta, d, dx_mir)
print("dtheta =",dtheta,",\n(z,y) = (",z,",",y,")")
# The number I'm interested in
dy_tol = y - y_0
# the associated z shifting.
dz_y_tol = z - z_0
print("(dz,dy_tol) = (", dz_y_tol, ",", dy_tol, ")")

dtheta = 0.036347621019 ,
(z,y) = ( 22.7299147859 , 24.3941039189 )
(dz,dy_tol) = ( 0.683789774835 , 2.3941039189 )


This change in x is close to our manipulatability in x to compensate. So, assuming we have to compensate fully, then we could only have half this range. That means:


In [234]:
dy_tol_eff = dy_tol / 2
print("dy_tol_eff =", dy_tol_eff)

dy_tol_eff = 1.19705195945


Note that there is an implicit approximation here that the flexibility range is the same in both the + and negative directions. This is only a good approximation around theta_0 = 45 degrees.

## Corresponding Angular Tolerances

### X Angular Tolerance

The x angular tolerance is easy to calculate, since it is decoupled from the y and z tolerances. The positional tolerance $dx_{tol}$ and the position $x_0$ correspond to some angle $\tan(2*\theta_{xtoll}) = \frac{dx_{toll}}{x_0}$, where the factor of 2 comes from the reflection of a beam based on picking up 2 times the angle because of angle of incidence = angle of reflection. Calculating this angle gives:

In [235]:
theta_x_tol = np.arctan(dx_tol / x_0) / 2
print("In radians:", theta_x_tol, "\nIn degrees:", theta_x_tol*180/np.pi)

In radians: 0.0181358205498 
In degrees: 1.03910597551


### YZ Angular Tolerance

I want to calculate an angular tolerance in the y,z angle that takes into account both the y tolerance and the z tolerance, slightly non-trivial. In general, I'm going to have some region of space around the center which is within my tolerance levels. I'm interested in the distance of the edge of this region, which can be given in terms of the x and y coordinates of this region:
r = sqrt(x^2+y^2)
for some points (x,y) on the perimeter of this region.
In this case my region is not a circle but an ellipse (since dy_tol_eff =/= dz_tol). I approximate this ellipse by having minor axis dz_tol and major axis dy_tol_eff. 

(Really the ellipse would not have major axis in the y direction and and minor axis in the z-direction. As I calculated earlier, trying to adjust in the +y direction automatically pushes the lattice intersection in the +z direction. We therefore have a larger tolerance in the +y,+z and -y,-z directions than we do in the +y,-z or -y,+z directions. The actual ellipse would then be rotated to some funny angle, but this is getting rather non-trivial to calculate so I take the minimum tolerance as being true in both +y,+z and +y,-z etc, approximating this as a nice ellipse. If really pushed for the tolerances, this more complicated calculation could be done to see what the limits would really be.)

The equation of this (nice, approximate) ellipse is

$(x/x_0)^2 + (y/y_0)^2 = 1 \rightarrow y^2 = y_0^2 * (1 - (x/x_0)^2)$
If I plug these numbers into my radius equation, then I get the radius of my ellipse as a function of x
$$r = \sqrt{x^2 + (y_0^2 * (1 - (x/x_0)^2))} = \sqrt{x^2 + y_0^2 - (y_0/x_0)^2*x^2} = \sqrt{x^2(1 - (y_0/x_0)^2)+y_0^2} = \sqrt{r^2*\cos^2(\theta)(1 - (y_0 / x_0)^2) + y_0^2}$$
$$r^2 = r^2*\cos^2(\theta)(1 - (y_0 / x_0)^2) + y_0^2$$
$$r^2(1-\cos^2(\theta)(1-(y_0 / x_0)^2) = y_0^2$$
$$r^2 = \frac{y_0^2}{(1 - \cos^2(\theta) * (1-(y_0/x_0)^2)}$$

If I then plug in my y and z tolerances and the angle I want to work on, then I get the total distance tolerance that I can have.

In [236]:
yz_tol = np.sqrt(dy_tol_eff**2 / (1 - (np.cos(theta_0))**2*(1-(dy_tol_eff / dz_tol)**2)))
print(yz_tol)

0.94026748154


Calculating the angle that corresponds to this distance tolerance using my previously define function:

In [237]:
theta_yz_tol = (angle_tolerance(theta_0, d, yz_tol) - theta_0)/2
print("in radians:", theta_yz_tol, "\nIn degrees:", theta_yz_tol * 180/np.pi)

in radians: [ 0.02131394] 
In degrees: [ 1.22119906]


## Conclusions

In [238]:
print("In conclusion, the angle tolerance in the x direction is +-{} "
      "degrees and my angle tolerance in the yz direction is +-{} degrees,"
      " but probabily a little better in the yz direction because of my "
      "approximation made earlier. Both of those are assuming "
      "a".format(theta_x_tol * 180 / np.pi, theta_yz_tol * 180/np.pi),
      "of {} mm in diameter, and would be higher for larger mirrors"
      ".".format(mirror_diameter))

In conclusion, the angle tolerance in the x direction is +-1.0391059755084762 degrees and my angle tolerance in the yz direction is +-[ 1.22119906] degrees, but probabily a little better in the yz direction because of my approximation made earlier. Both of those are assuming a of 2 mm in diameter, and would be higher for larger mirrors.
